In [1]:
import pydpf
import torch
from torch import Tensor
from typing import Tuple, Union
from pydpf.datautils import StateSpaceDataset
import explicit_model

from pydpf.distributions.Gaussian import MultivariateGaussian, LinearGaussian
from pathlib import Path
import numpy as np
import training

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
data_folder =  Path('./data/test/')

## Utility functions

In [2]:
#set the seed
gen_num = 9

#Easy way to create new generator objects to manage RNG
def new_gen():
    global gen_num
    global device
    gen_num += 1
    return torch.Generator(device=device).manual_seed(gen_num)

#Functions to create random matrices for parameter initialisation
def get_spectral_radius(M):
    eigvals = torch.linalg.eigvals(M)
    return torch.max(torch.abs(eigvals))

def make_random(size, range, sparsity, device, generator):
    return torch.where(torch.rand(size, device=device, generator=generator) < sparsity, torch.rand(size, device=device, generator=generator) * (range[1] - range[0]) + range[0], 0) 

def make_random_matrix(*, data:Tensor=None, device:torch.device=None, force_diagonal:bool=False, size:Union[Tuple[int,int], int]=None, range:Tuple[float,float]=(0.,1.), diag_range:Tuple[float,float]=(0.,1.), off_diag_range:Tuple[float,float]=(0., 1.), max_radius:float=None, generator:torch.Generator=None, positive_definite:bool=False, requires_grad:bool = True, sparsity:float=1.):
    if generator is None:
        generator = torch.Generator(device=device)
    if not data is None:
        return torch.nn.Parameter(data, requires_grad)
    if isinstance(size, int):
        vec = make_random(size, range, sparsity, device, generator)
        return torch.nn.Parameter(vec, requires_grad)
    if size[0]==size[1]:
        if positive_definite and diag_range[0]<0:
            raise ValueError("Diagonal range must be positive for positive definite matrices")
        
        diag = make_random(size[0], diag_range, sparsity, device, generator)
        matrix = torch.diag(diag)
        if not force_diagonal:
            off_diag = make_random(size, off_diag_range, sparsity, device, generator) * (1-torch.eye(size[0], device=device))
            matrix += off_diag
            if positive_definite:
                matrix = matrix.T @ matrix
        if max_radius is not None:
            radius = get_spectral_radius(matrix)
            if radius > max_radius:
                raise ValueError(f'Spectral radius {radius} exceeds maximum {max_radius}, consider decreasing the range or trying a different seed')
        return torch.nn.Parameter(matrix, requires_grad)
            
    matrix = make_random(size, range, sparsity, device, generator)
    return torch.nn.Parameter(matrix, requires_grad)

def load_matrix_csv(path, name):
    matrix_loc = path / name
    return torch.tensor(np.loadtxt(matrix_loc, delimiter=','), device=device, dtype=torch.float32)
    

In [3]:
data_loc = data_folder / 'data.csv'
normalise = True

#Load the data
data_set = StateSpaceDataset(data_loc, state_prefix='state', device=device)
initialisation_generator = new_gen()

## Parameter initialisation

In [4]:
#Initialise model parameters as random Matrices
prior_covariance = make_random_matrix(size = (data_set.state_dimension, data_set.state_dimension), diag_range=(0, 1), off_diag_range=(-1, 1), generator=initialisation_generator, device=device, positive_definite=True)
dynamic_matrix = make_random_matrix(size=(data_set.state_dimension,data_set.state_dimension), diag_range=(-0.2, 0.2), off_diag_range=(-0.3, 0.3), generator=initialisation_generator, max_radius=1, device=device)
dynamic_bias = make_random_matrix(size=data_set.state_dimension, range=(-0.3, 0.3), generator=initialisation_generator, device=device, requires_grad=False)
dynamic_covariance = make_random_matrix(size = (data_set.state_dimension, data_set.state_dimension), diag_range=(0, 1), off_diag_range=(-1, 1), generator=initialisation_generator, device=device, positive_definite=True)
if data_set.state_dimension==data_set.observation_dimension:
    observation_matrix = make_random_matrix(size=(data_set.observation_dimension,data_set.state_dimension), diag_range=(-1, 1), off_diag_range=(-1, 1), generator=initialisation_generator, device=device)
else:
    observation_matrix = make_random_matrix(size=(data_set.observation_dimension,data_set.state_dimension), range=(-1, 1), generator=initialisation_generator, device=device)
observation_bias = make_random_matrix(size=data_set.state_dimension, range=(-1, 1), generator=initialisation_generator, device=device)
observation_covariance = make_random_matrix(size = (data_set.observation_dimension, data_set.observation_dimension), diag_range=(0, 1), off_diag_range=(-1, 1), generator=initialisation_generator, device=device, positive_definite=True)

#Take the Cholesky decomposition of the covariances and set it in place to avoid adding to the computation graph
prior_covariance.data = torch.linalg.cholesky(prior_covariance)
dynamic_covariance.data = torch.linalg.cholesky(dynamic_covariance)
observation_covariance.data = torch.linalg.cholesky(observation_covariance)




## Defining the Model

In [5]:
#Define the model components as distribution objects
prior_dist = MultivariateGaussian(mean = torch.zeros(data_set.state_dimension, device = device), cholesky_covariance=prior_covariance, generator=new_gen())
dynamic_dist = LinearGaussian(weight = dynamic_matrix, bias = dynamic_bias, cholesky_covariance=dynamic_covariance, generator=new_gen(), constrain_spectral_radius=0.99)
observation_dist = LinearGaussian(weight = observation_matrix, bias = observation_bias, cholesky_covariance=observation_covariance, generator=new_gen())

#Register model componets as a FilteringModel
SSM = pydpf.FilteringModel(dynamic_model=dynamic_dist, observation_model=observation_dist, prior_model=prior_dist)
#Define a DPF to run the FilteringModel
dpf = pydpf.DPF(SSM, new_gen())

#kernel = pydpf.KernelMixture([('Gaussian', data_set.state_dimension)], generator=new_gen(), gradient_estimator='reparameterisation')
#dpf = pydpf.KernelDPF(SSM, kernel)
#dpf = pydpf.SVGDKernelDPF(SSM, kernel, iterations=10, lr=1, alpha=0.9)

## Defining the model with generic API

In [6]:
#Define the model components as custom Modules
prior_dist = explicit_model.GaussianPrior(mean = torch.zeros(data_set.state_dimension, device = device), cholesky_covariance=prior_covariance, generator=new_gen(), device=device)
dynamic_dist = explicit_model.LinearGaussianDynamic(weight = dynamic_matrix, bias = dynamic_bias, cholesky_covariance=dynamic_covariance, device = device, generator=new_gen(), max_spectral_radius=0.99)
observation_dist = explicit_model.LinearGaussianObservation(weight = observation_matrix, bias = observation_bias, cholesky_covariance=observation_covariance, device = device, generator=new_gen())

#Register model componets as a FilteringModel
SSM = pydpf.FilteringModel(dynamic_model=dynamic_dist, observation_model=observation_dist, prior_model=prior_dist)
#Define a DPF to run the FilteringModel
dpf = pydpf.DPF(SSM, new_gen())

## Training

In [8]:
#Train the model with a fairly standard torch training loop.
#Use the Adam optimser; MSE loss; and clip the gradients elementwise each iteration.
training.train(dpf, torch.optim.Adam(dpf.parameters(), lr=1e-2), data_set, 50, (50, 50, 1000), (30, -1, -1), (0.5, 0.25, 0.25), pydpf.MSE_Loss(), None, torch.Generator().manual_seed(0), pydpf.ClipByElement(1.))
for n, p in dpf.named_parameters():
    print(n)
    print(p)

TypeError: GaussianPrior.sample() got an unexpected keyword argument 't'

## Load the true data generating model

In [11]:
#kernel = pydpf.KernelMixture([('Gaussian', data_set.state_dimension)], generator=new_gen(), gradient_estimator='reparameterisation')
prior_covariance = load_matrix_csv(data_folder, 'prior_covariance.csv')
dynamic_covariance = load_matrix_csv(data_folder, 'dynamic_covariance.csv')
dynamic_bias = load_matrix_csv(data_folder, 'dynamic_bias.csv')
dynamic_matrix = load_matrix_csv(data_folder, 'dynamic_matrix.csv')
observation_bias = load_matrix_csv(data_folder, 'observation_bias.csv')
observation_covariance = load_matrix_csv(data_folder, 'observation_covariance.csv')
observation_matrix = load_matrix_csv(data_folder, 'observation_matrix.csv')
prior_dist = MultivariateGaussian(mean = torch.zeros(data_set.state_dimension, device = device), cholesky_covariance=torch.linalg.cholesky(prior_covariance), generator=new_gen())
dynamic_dist = LinearGaussian(weight = dynamic_matrix, bias = dynamic_bias, cholesky_covariance=torch.linalg.cholesky(dynamic_covariance), generator=new_gen(), constrain_spectral_radius=0.99)
observation_dist = LinearGaussian(weight = observation_matrix, bias = observation_bias, cholesky_covariance=torch.linalg.cholesky(observation_covariance), generator=new_gen())
SSM = pydpf.FilteringModel(dynamic_model=dynamic_dist, observation_model=observation_dist, prior_model=prior_dist)
pf = pydpf.DPF(SSM, resampling_generator=new_gen())
#pf = pydpf.SVGDKernelDPF(SSM, kernel, iterations=10, lr=1, alpha=0.9)

## Run the true model

In [12]:
training.test(pf, data_set, 1000, 20, pydpf.MSE_Loss(), data_loading_generator=torch.Generator().manual_seed(10))

   
loss = 0.19143067836761474
